# 'PBULISHED:, UPDATED:' 2차제거  && 특수문자제거 및 소문자만들기

In [2]:
import unicodedata
import re
import numpy as np
import pandas as pd
from os import listdir

result = pd.read_csv('dailymail_stories_preprocessing1.csv')

# 해당값이 아닌것만 저장! 즉, 해당값 제거!
# 변수에 따로 넣어줘야함!
# result = result[result.index != 1]
result[result.index != 1]

print(len(result['story']))
print(len(result['highlights']))

120411
120411


In [3]:
# 특수문자 제거 & 소문자 만들기
def preprocess_sentence(line):

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    line = re.sub(r"([?.!,¿])", r" \1 ", line)
    line = re.sub(r'[" "]+', " ", line)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    line = re.sub(r"[^a-zA-Z0-9?.!,@%$¿]+", " ", line)

    return line.strip().lower()




# PUBLISHED: & UPDATED: 2차 삭제!!!!!!!!
useless_list = []
for i, v in enumerate(result['story']):
    if 'PUBLISHED:' in v or 'UPDATED:' in v :
        useless_list.append(i)
        result = result[result.index != i]
    else:
        result['story'][i] = preprocess_sentence(result['story'][i])
        result['highlights'][i] = preprocess_sentence(result['highlights'][i])

# 120,411 - (726 + 86) = 119,599
print(len(useless_list))
print(len(result['story']))
print(len(result['highlights']))
result.to_csv('dailymail_stories_preprocessing2.csv', index=False)

812
119599
119599


# 중복값 체크1 (중복 제거 후 필요한만큼 데이터 저장)

In [200]:
duplicate_check = pd.read_csv('dailymail_stories_preprocessing2.csv').dropna()
print('중복체크 이전')
print(len(duplicate_check['story']))
print(len(duplicate_check['highlights']), '\n')

duplicate_check = duplicate_check.drop_duplicates(subset = 'story')
print('story 중복체크 후 데이터 갯수')
print(len(duplicate_check['story']))
print(len(duplicate_check['highlights']), '\n')

duplicate_check = duplicate_check.drop_duplicates(subset = 'highlights')
print('highlights 중복체크 후 데이터 갯수')
print(len(duplicate_check['story']))
print(len(duplicate_check['highlights']))

duplicate_check.to_csv('dailymail_stories_preprocessing2.csv', index = False)   

중복체크 이전
119599
119599 

story 중복체크 후 데이터 갯수
119576
119576 

highlights 중복체크 후 데이터 갯수
119151
119151


# dailymail 데이터 랜덤하게 섞고 6만개 데이터 저장(train+vali)

In [201]:
dailymail_articles = pd.read_csv('dailymail_stories_preprocessing2.csv')
print(len(dailymail_articles['story']))   #119151

# The frac keyword argument specifies the fraction of rows to return in the random sample, so frac=1 means return all rows (in random order).
# Here, specifying drop=True prevents .reset_index from creating a column containing the old index entries.
dailymail_articles = dailymail_articles.sample(frac=1).reset_index(drop=True)
dailymail_articles = dailymail_articles[:60000]
dailymail_articles.to_csv('dailymail_stories.csv', index=False)

119151


# train, test 분류(5만, 1만)

In [202]:
dailymail_articles = pd.read_csv('dailymail_stories.csv')
dailymail_articles_train = dailymail_articles[:50000]
dailymail_articles_train.to_csv('dailymail_stories_train.csv', index=False)

dailymail_articles_test = dailymail_articles[50000:]
dailymail_articles_test.to_csv('dailymail_stories_test.csv', index=False)

print(len(dailymail_articles_train['highlights']))
print(len(dailymail_articles_test['story']))

50000
10000


# CNN, DAILYMAIL train_data 합치기!

In [209]:
cnn_train = pd.read_csv('cnn_stories_train.csv').dropna()
dailymail_train = pd.read_csv('dailymail_stories_train.csv').dropna()

print(len(cnn_train['story']))
print(len(cnn_train['highlights']))
print(len(dailymail_train['story']))
print(len(dailymail_train['highlights']))

50000
50000
50000
50000


In [210]:
train_data = cnn_train.append(dailymail_train, ignore_index = True)   #index값 순차적으로 설정! (0-99999)
print(train_data)

                                              highlights  \
0      the american public must be part of preventing...   
1      java vulnerability is due to incomplete patch ...   
2      netanyahu s party wins but the centrists do we...   
3      ruben navarrette we re a nation divided into t...   
4      pastels and prints among recurring trends at t...   
5      1 , 500 protest effort to end birthright citiz...   
6      former secretary of state u . s . made huge mi...   
7      flooding in pakistan has left more than 420 pe...   
8      catfish creators say play it safe when meeting...   
9      the usda is considering assigning veterinarian...   
10     nasa used company that specializes in restorin...   
11     high temperatures are recorded on the northern...   
12     despite split , there is no word on a divorce ...   
13     paul waldman background check failure another ...   
14     michael hossack lost his battle with cancer on...   
15     google s project wing has flown m

In [211]:
train_data.to_csv('train_data.csv', index=False)

# CNN, DAILYMAIL test_data 합치기!

In [212]:
cnn_test = pd.read_csv('cnn_stories_test.csv').dropna()
dailymail_test = pd.read_csv('dailymail_stories_test.csv').dropna()

print(len(cnn_test['story']))
print(len(cnn_test['highlights']))
print(len(dailymail_test['story']))
print(len(dailymail_test['highlights']))

10000
10000
10000
10000


In [213]:
test_data = cnn_test.append(dailymail_test, ignore_index = True)   #index값 순차적으로 설정! (0-99999)
print(test_data)

                                              highlights  \
0      mohamed morsy , in berlin , rejects extraordin...   
1      uruguay thrash jordan 5 0 in their world cup q...   
2      sen . barack obama , iraqi prime minister repo...   
3      an indiana father and son s dreams of raising ...   
4      rules let groups like religious hospitals , sc...   
5      ryan calo like pilot s claim , we are going to...   
6      iranian conservatives say they captured most s...   
7      u . s . open champion sam stosur loses to unse...   
8      japanese football team vegalta sendai had best...   
9      united states is proposing tough new sanctions...   
10     two pharmacies sold drugs without valid prescr...   
11     surely the president and his staff will make t...   
12     unabomber ted kaczynski fighting to stop aucti...   
13     men suffered injuries after being carried alon...   
14     australian expert panel recommends opening off...   
15     pele threatened with court action

In [214]:
test_data.to_csv('test_data.csv', index=False)

# 전체 중복값 체크

In [194]:
#예시
data = pd.DataFrame({'key1':['a', 'b', 'b', 'c', 'c'], 'key2':['v', 'w', 'w', 'x', 'y'], 'col':[1, 2, 3, 4, 5]})
print(len(data['key1']))   # 중복값 체크 없을 땐 5개

data = data.drop_duplicates(subset = 'key1')
print(len(data['key1']))   # 중복 체크하면 3개
print('\n', data)

data.to_csv('aa.csv', index = False)   
gg = pd.read_csv('aa.csv')
print('\n', gg)

  key1 key2  col
0    a    v    1
1    b    w    2
2    b    w    3
3    c    x    4
4    c    y    5 

5
3

   key1 key2  col
0    a    v    1
1    b    w    2
3    c    x    4

   key1 key2  col
0    a    v    1
1    b    w    2
2    c    x    4


In [218]:
check = pd.read_csv('train_data.csv').dropna()
print(len(check['highlights']))    
print(len(check['highlights']))   
data = check.drop_duplicates(subset = 'highlights')
print(len(data['highlights']))
print(len(data['highlights']))   

100000
100000
99999
99999


# 데이터 길이

In [98]:
#story = pd.read_csv('test_data.csv').dropna()
story = pd.read_csv('train_data.csv').dropna()

h_count_list = []
for h in story['story']:
    c = len(h.split(' '))
    h_count_list.append(c)
    
    if c ==14:
        print(h)


h_count_list.sort()
print(min(h_count_list))
print(max(h_count_list))
print(len(h_count_list))
print(sum(h_count_list)/len(h_count_list))
print(h_count_list[:50])

congressman jared polis d colorado district 02 congressman jason chaffetz r utah district 03
14
2304
100000
757.25586
[14, 26, 34, 39, 40, 46, 46, 48, 48, 49, 51, 53, 53, 54, 55, 56, 57, 59, 59, 60, 60, 64, 64, 64, 65, 66, 66, 68, 69, 69, 70, 71, 72, 72, 72, 73, 73, 73, 74, 74, 74, 75, 75, 75, 76, 76, 76, 78, 80, 80]
